In [1]:
#Generating the Statistical significant growth in assertions

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
from copy import deepcopy
from SPARQLWrapper import SPARQLWrapper, JSON, POSTDIRECTLY
from utils import MatrixIO
mfio = MatrixIO()

In [ ]:
def get_simple_results_normal(endpoint, query, limres = False):
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(3600000)
    #sparql.setCredentials("admin", "admin")
    if limres: sparql.addExtraURITag("limit", "1000")
    try: 
        results = sparql.query().convert()
        #print results
        #results = process_xml_results(results)
    except Exception:
        print "Error encountered for " + query
        return None
    return results

def _idquery(iden):
    q1 = "SELECT DISTINCT ?p ?o WHERE {{<"+iden+"> ?p ?o} UNION {?o ?p <"+iden+">}}"
    results = get_simple_results_normal(spq, q1)
    res_pair = {}
    for res in results["results"]["bindings"]:
        if not res["p"]["value"] in res_pair: res_pair[res["p"]["value"]] = set([])
        res_pair[res["p"]["value"]].add(res["o"]["value"])
    return res_pair

def _ebiquery(iden):
    #q1 = "PREFIX skos: <http://www.w3.org/2004/02/skos/core#> 
    #   SELECT DISTINCT ?p ?o WHERE {?x skos:exactMatch <" +iden + ">. {{?x ?p ?o} UNION {?o ?p ?x}}}"
    #print q1
    q1 = "SELECT DISTINCT ?p ?o WHERE {{<"+iden+"> ?p ?o} UNION {?o ?p <"+iden+">}}"
    results = get_simple_results_normal(uniprotq, q1, limres=True)
    print iden, len(results["results"]["bindings"])
    res_pair = {}
    for res in results["results"]["bindings"]:
        if not res["p"]["value"] in res_pair: res_pair[res["p"]["value"]] = set([])
        res_pair[res["p"]["value"]].add(res["o"]["value"])
    return res_pair

def _ebiquery_col(col, _df=drug_ids):
    _set = {}
    ccount = 0
    unique_pred = set([])
    for k in list(_df[col]):
        ccount +=1
        cbidp = k.split(":")
        #cbid = "http://purl.obolibrary.org/obo/CHEBI_" + cbidp[len(cbidp)-1]
        #cbid = "http://rdf.ebi.ac.uk/resource/ensembl/" + cbidp[len(cbidp)-1]
        cbid = "http://purl.uniprot.org/uniprot/" + cbidp[len(cbidp)-1]
        _set[k] = _ebiquery(cbid)
        unique_pred = unique_pred.union(_set[k].keys())
        print k, ccount, len(_set[k])
    return _set, unique_pred

def _query_col(col, _df=drug_ids):
    _set = {}
    ccount = 0
    unique_pred = set([])
    for k in list(_df[col]):
        ccount +=1
        _set[k] = _idquery(k)
        unique_pred = unique_pred.union(_set[k].keys())
        print k, ccount, len(_set[k])
    return _set, unique_pred

In [ ]:
def getiddict(_ids):
    _ids_dict = _ids.set_index("x")
    _ids_dict = _ids_dict.to_dict(orient="index")
    return _ids_dict

def get_base(_data):
    _base = {}
    for k in _data:
        if not k in _base: _base[k] = {}
        for m in _data[k]:
            if m in allowed_preds:
                _base[k][m] = _data[k][m]
    return _base

def append_base(_data, _ids, dtype, _oddata):
    upddata = deepcopy(_data)
    for k in _data:
        #print _ids[k][dtype]
        if not _ids[k][dtype] in _oddata: continue
        m = _ids[k][dtype]
        for n in _oddata[m]:
            if not n in allowed_preds: continue
            upddata[k][n] = _oddata[m][n]
    return upddata

def get_box_data(_data, dset_name, dtype):
    bx = {}
    for k in _data:
        ccount = 0
        for m in _data[k]: ccount += len(_data[k][m])
        bx[k] = {"count": ccount, "dset": dset_name, "type": dtype}
    return bx

In [ ]:
ebiq = "https://www.ebi.ac.uk/rdf/services/sparql"
uniprotq = "http://sparql.uniprot.org/sparql"
spq = "http://bio2rdf.org/sparql/"
#_ebiquery("http://purl.obolibrary.org/obo/CHEBI_46195")

In [ ]:
#--------------------------------------------------------------------------

In [ ]:
#Read all drug identifiers and query each individually against each source ...
drug_ids = pd.read_csv("sparql1.csv", sep=",")
drug_ids.head()

In [ ]:
drugbank_data, drugbank_preds = _query_col('x')
kegg_data, kegg_preds = _query_col("kegg")
pharmgkb_data, pharmgkb_preds = _query_col("pharmgkb")
ctd_data, ctd_preds = _query_col("mesh")
chembl_data, chembl_preds = _ebiquery_col("chebi")
#for k in kegg_preds: print k

In [ ]:
mfio.save_matrix(chembl_data, "chembl_data.dat")
mfio.save_matrix(drugbank_data, "drugbank_data.dat")
mfio.save_matrix(kegg_data, "kegg_data.dat")
mfio.save_matrix(pharmgkb_data, "pharmgkb_data.dat")
mfio.save_matrix(ctd_data, "ctd_data.dat")

In [2]:
#--------------------------------------------------------------------------

In [ ]:
dise_ids = pd.read_csv("sparql2.csv", sep=",")
dise_ids.head()

In [ ]:
keggdise_data, keggdise_preds = _query_col("x", dise_ids)
ctddise_data, ctddise_preds = _query_col("mesh", dise_ids)
pgkdise_data, pgkdise_preds = _query_col("pharmgkb", dise_ids)

In [ ]:
mfio.save_matrix(keggdise_data, "keggdise_data.dat")
mfio.save_matrix(pgkdise_data, "pharmgkbdise_data.dat")
mfio.save_matrix(ctddise_data, "ctddise_data.dat")

In [ ]:
#--------------------------------------------------------------------------

In [ ]:
protein_ids = pd.read_csv("sparql3.csv", sep=",")
protein_ids.head()

In [ ]:
drugbankprot_data, drugbankprot_preds = _query_col('x', protein_ids)
keggprot_data, keggprot_preds = _query_col('kegg', protein_ids)
pharmgkbprot_data, pharmgkbprot_preds = _query_col('pharmgkb', protein_ids)
ctdprot_data, ctdprot_preds = _query_col('ncbigene', protein_ids)
ensembl_data, ensembl_preds = _ebiquery_col("ensembl", protein_ids)
uniprot_data, uniprot_preds = _ebiquery_col("uniprot", protein_ids)

In [ ]:
mfio.save_matrix(ensembl_data, "ensembl_data.dat")
mfio.save_matrix(uniprot_data, "uniprot_data.dat")
mfio.save_matrix(drugbankprot_data, "drugbankprot_data.dat")
mfio.save_matrix(keggprot_data, "keggprot_data.dat")
mfio.save_matrix(pharmgkbprot_data, "pharmgkbprot_data.dat")
mfio.save_matrix(ctdprot_data, "ctdprot_data.dat")

In [ ]:
#--------------------------------------------------------------------------

In [ ]:
drug_ids_dict = getiddict(drug_ids)
dise_ids_dict = getiddict(dise_ids)
prot_ids_dict = getiddict(protein_ids)

In [ ]:
all_preds = set([])
all_preds = all_preds.union(ensembl_preds)
all_preds = all_preds.union(uniprot_preds)
all_preds = all_preds.union(drugbankprot_preds)
all_preds = all_preds.union(keggprot_preds)
all_preds = all_preds.union(ctdprot_preds)
all_preds = all_preds.union(pharmgkbprot_preds)
all_preds = all_preds.union(pgkdise_preds)
all_preds = all_preds.union(keggdise_preds)
all_preds = all_preds.union(ctddise_preds)
all_preds = all_preds.union(pharmgkb_preds)
all_preds = all_preds.union(drugbank_preds)
all_preds = all_preds.union(kegg_preds)
all_preds = all_preds.union(ctd_preds)
all_preds = all_preds.union(chembl_preds)

In [ ]:
all_predsf = open("all_predsasda.csv", "w+")
for k in all_preds: all_predsf.write(k + "\n")
all_predsf.close()

In [ ]:
#select the predicates to use from mapping rules/CANCO .....
allowed_preds = pd.read_csv("all_preds1.csv", sep="\t", header=None)
allowed_preds = list(allowed_preds[0])
print len(allowed_preds)

In [ ]:
#--------------------------------------------------------------------------

In [ ]:
drugs_data = {}
prots_data = {}
dises_data = {}

drugs_data = get_base(drugbank_data)
prots_data = get_base(drugbankprot_data)
dises_data = get_base(keggdise_data)

In [ ]:
drugs1_data = append_base(drugs_data, drug_ids_dict, "kegg", kegg_data)
drugs2_data = append_base(drugs1_data, drug_ids_dict, "pharmgkb", pharmgkb_data)
drugs3_data = append_base(drugs2_data, drug_ids_dict, "mesh", ctd_data)
drugs4_data = append_base(drugs3_data, drug_ids_dict, "chebi", chembl_data)

In [ ]:
prots1_data = append_base(prots_data, prot_ids_dict, "kegg", keggprot_data)
prots2_data = append_base(prots1_data, prot_ids_dict, "pharmgkb", pharmgkbprot_data)
prots3_data = append_base(prots2_data, prot_ids_dict, "ncbigene", ctdprot_data)
prots4_data = append_base(prots3_data, prot_ids_dict, "ensembl", ensembl_data)
prots5_data = append_base(prots4_data, prot_ids_dict, "uniprot", uniprot_data)

In [ ]:
dises1_data = append_base(dises_data, dise_ids_dict, "pharmgkb", pgkdise_data)
dises2_data = append_base(dises1_data, dise_ids_dict, "mesh", ctddise_data)

In [ ]:
#--------------------------------------------------------------------------

In [ ]:
bx1 = get_box_data(drugs_data, "1-DrugBank", "Drug")
bx2 = get_box_data(drugs1_data, "2-KEGG", "Drug")
bx3 = get_box_data(drugs2_data, "3-PharmGKB", "Drug")
bx4 = get_box_data(drugs3_data, "4-CTD", "Drug")
bx5 = get_box_data(drugs4_data, "5-EBIRDF", "Drug")

In [ ]:
pbx1 = get_box_data(prots_data, "1-DrugBank", "Protein")
pbx2 = get_box_data(prots1_data, "2-KEGG", "Protein")
pbx3 = get_box_data(prots2_data, "3-PharmGKB", "Protein")
pbx4 = get_box_data(prots3_data, "4-CTD", "Protein")
pbx5 = get_box_data(prots4_data, "5-EBIRDF", "Protein")
pbx6 = get_box_data(prots5_data, "6-UniProt", "Protein")

In [ ]:
dbx2 = get_box_data(dises_data, "2-KEGG", "Disease")
dbx3 = get_box_data(dises1_data, "3-PharmGKB", "Disease")
dbx4 = get_box_data(dises2_data, "4-CTD", "Disease")

In [ ]:
#--------------------------------------------------------------------------

In [ ]:
cbound = pd.DataFrame.from_dict(bx1, orient="index")
cbound = cbound.reset_index()
cbound.head()

In [ ]:
def _append_bx(bx, cbound):
    cbound1 = pd.DataFrame.from_dict(bx, orient="index")
    cbound1 = cbound1.reset_index()
    cbound = pd.concat([cbound, cbound1], axis=0)
    print cbound.shape
    return cbound

def _append_blankbx(_base, cbound, iden, dtype):
    bx = {}
    for k in _base:
        bx[k] = {"count": _base[k]["count"], "dset": iden, "type": dtype}
    cbound1 = pd.DataFrame.from_dict(bx, orient="index")
    cbound1 = cbound1.reset_index()
    cbound = pd.concat([cbound, cbound1], axis=0)
    print cbound.shape
    return cbound
    
cbound = _append_bx(bx2, cbound) 
cbound = _append_bx(bx3, cbound) 
cbound = _append_bx(bx4, cbound) 
cbound = _append_bx(bx5, cbound) 
cbound = _append_bx(pbx1, cbound) 
cbound = _append_bx(pbx2, cbound) 
cbound = _append_bx(pbx3, cbound) 
cbound = _append_bx(pbx4, cbound)
cbound = _append_bx(pbx5, cbound) 
cbound = _append_bx(pbx6, cbound)
cbound = _append_bx(dbx2, cbound) 
cbound = _append_bx(dbx3, cbound) 
cbound = _append_bx(dbx4, cbound)

In [ ]:
#wait here ---- 
#cbound = _append_blankbx(drugs4_data, "6-UniProt", "Drug")

In [ ]:
cbound.to_csv("cbound_agg.tsv", sep="\t", index=None)

In [ ]:
for k in bx1: print k, bx1[k]["count"], bx2[k]["count"], bx3[k]["count"], bx4[k]["count"], bx5[k]["count"]
for k in pbx1: print k, pbx1[k]["count"], pbx2[k]["count"], pbx3[k]["count"], pbx4[k]["count"], pbx5[k]["count"], pbx6[k]["count"]
for k in dbx2: print k, dbx2[k]["count"], dbx3[k]["count"], dbx4[k]["count"]

In [ ]:
# Check R plot for more .... 